In [1]:
from IPython import display
from ipywidgets import Image

import os
import datajoint as dj

In [2]:
schema = dj.schema('test_filepath')

Connecting dimitri@localhost:3306


In [7]:
## Storage configuration

# set up stores
dj.config['stores'] = {
    'remote': dict(   # store in minio
        stage=os.path.abspath('./stage'),
        protocol='s3',
        endpoint='localhost:9000',
        access_key='datajoint',
        secret_key='datajoint',
        bucket='datajoint-demo',        
        location='dj/file-demo'),   
    }

## Download some images off the web into ./stage

In [3]:
# Step 1: Find a bunch of images on the web
logos = dict(
    ucsd='https://upload.wikimedia.org/wikipedia/commons/f/f6/UCSD_logo.png',
    datajoint='https://datajoint.io/static/images/DJiotitle.png',
    utah='https://umc.utah.edu/wp-content/uploads/sites/15/2015/01/Ulogo_400p.png',
    bcm='https://upload.wikimedia.org/wikipedia/commons/5/5d/Baylor_College_of_Medicine_Logo.png',
    pydata='https://pydata.org/wp-content/uploads/2018/10/pydata-logo.png',
    python='https://www.python.org/static/community_logos/python-logo-master-v3-TM.png',
    pni='https://vathes.com/2018/05/24/Princeton-Neuroscience-Institute-Partners-with-Vathes-to-Support-the-Adoption-of-DataJoint/PNI%20logo.png')

In [4]:
@schema
class Organization(dj.Lookup):
    definition = """
    organization : varchar(30)
    --- 
    logo_url : varchar(255)
    """
    contents = logos.items()

In [5]:
Organization()

organization,logo_url
bcm,https://upload.wikimedia.org/wikipedia/commons/5/5d/Baylor_College_of_Medicine_Logo.png
datajoint,https://datajoint.io/static/images/DJiotitle.png
pni,https://vathes.com/2018/05/24/Princeton-Neuroscience-Institute-Partners-with-Vathes-to-Support-the-Adoption-of-DataJoint/PNI%20logo.png


In [8]:
import requests 

@schema
class Logo(dj.Imported):
    definition = """
    -> Organization
    ---
    logo_image : filepath@remote
    """
    
    path = os.path.join(dj.config['stores']['remote']['stage'], 'organizations', 'logos')
    
    def make(self, key):
        os.makedirs(self.path, exist_ok=True)
        url = (Organization & key).fetch1('logo_url')
        local_file = os.path.join(self.path, key['organization'] + os.path.splitext(url)[1])
        print(local_file)
        with open(local_file, 'wb') as f:
            f.write(requests.get(url).content)
        self.insert1(dict(key, logo_image=local_file))        

In [25]:
Logo.populate()

/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/bcm.png
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/datajoint.png


In [10]:
Logo()

organization,logo_image
pydata,=BLOB=
utah,=BLOB=
datajoint,=BLOB=


In [11]:
Logo.fetch(as_dict=True)

[{'organization': 'pydata',
  'logo_image': '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/pydata.png'},
 {'organization': 'utah',
  'logo_image': '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/utah.png'},
 {'organization': 'datajoint',
  'logo_image': '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/datajoint.png'},
 {'organization': 'ucsd',
  'logo_image': '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/ucsd.png'},
 {'organization': 'pni',
  'logo_image': '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/pni.png'},
 {'organization': 'python',
  'logo_image': '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/python.png'},
 {'organization': 'bcm',
  'logo_image': '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/bcm.png'}]

In [12]:
# delete the local repository completely
import shutil
shutil.rmtree(dj.config['stores']['remote']['stage'])

In [13]:
paths = Logo().fetch('logo_image')

In [14]:
paths

array(['/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/pydata.png',
       '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/utah.png',
       '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/datajoint.png',
       '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/ucsd.png',
       '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/pni.png',
       '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/python.png',
       '/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/organizations/logos/bcm.png'],
      dtype=object)

In [18]:
Image.from_file(paths[5])

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02Y\x00\x00\x00\xcb\x08\x06\x00\x00\x00]\xc9\x86&\x…

In [19]:
(Logo & 'organization in ("datajoint", "bcm")').delete()

About to delete:
`test_filepath`.`_logo`: 2 items
Proceed? [yes, No]: yes
Committed.


In [20]:
ext = schema.external['remote']

In [21]:
ext.delete()

Deleted 2 items


In [23]:
ext.clean_filepaths()

Finding untracking files...
Deleting...
Done


In [24]:
list(ext.get_untracked_filepaths())

[]

In [26]:
states = dict(
    AL='Alabama',        AK='Alaska',       AZ='Arizona',      AR='Arkansas',
    CA='California',     CO='Colorado',     CT='Connecticut',  DE='Delaware',
    FL='Florida',        GA='Georgia',      HI='Hawaii',       ID='Idaho', 
    IL='Illinois',       IN='Indiana',      IA='Iowa',         KS='Kansas',
    KY='Kentucky',       LA='Louisiana',    ME='Maine',        MD='Maryland',
    MA='Massachusetts',  MI='Michigan',     MN='Minnesota',    MS='Mississippi',
    MO='Missouri',       MT='Montana',      NE='Nebraska',     NV='Nevada',
    NH='New Hampshire',  NJ='New Jersey',   NM='New Mexico',   NY='New York',
    NC='North Carolina', ND='North Dakota', OH='Ohio',         OK='Oklahoma',
    OR='Oregon',         PA='Pennsylvania', RI='Rhode Island', SC='South Carlina',
    SD='South Dakota',   TN='Tennessee',    TX='Texas',        UT='Utah',
    VT='Vermont',        VA='Virginia',     WA='Washington',   WV='West Virginia', 
    WI='Wisconsin',      WY='Wyoming')

In [27]:
@schema
class State(dj.Lookup):
    definition = """
    # United States
    state_code : char(2)
    ---
    state : varchar(20)
    """
    contents = states.items()

In [28]:
State()

state_code,state
AK,Alaska
AL,Alabama
AR,Arkansas


In [29]:
@schema
class StateBird(dj.Imported):
    definition = """
    -> State
    ---
    bird_image : filepath@remote    
    """
    path = os.path.join(dj.config['stores']['remote']['stage'], 'states', 'birds')
    
    
    def make(self, key):
        os.makedirs(self.path, exist_ok=True)
        state = (State & key).fetch1('state')
        url = "http://www.theus50.com/images/state-birds/{state}-bird.jpg".format(state=state.lower())
        local_file = os.path.join(self.path, state.lower() + os.path.splitext(url)[1])
        print(local_file)
        with open(local_file, 'wb') as f:
            f.write(requests.get(url).content)
        self.insert1(dict(key, bird_image=local_file))        


In [30]:
StateBird.populate()

/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/alaska.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/alabama.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/arkansas.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/arizona.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/california.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/colorado.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/connecticut.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/delaware.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/florida.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/georgia.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/birds/hawaii.jpg
/home/dimitri

In [31]:
@schema
class StateFlower(dj.Imported):
    definition = """
    -> State
    ---
    flower_image : filepath@remote    
    """
    path = os.path.join(dj.config['stores']['remote']['stage'],'states', 'flowers')
    
    
    def make(self, key):
        os.makedirs(self.path, exist_ok=True)
        state = (State & key).fetch1('state')
        url = "http://www.theus50.com/images/state-birds/{state}-flower.jpg".format(state=state.lower())
        local_file = os.path.join(self.path, state.lower() + os.path.splitext(url)[1])
        print(local_file)
        with open(local_file, 'wb') as f:
            f.write(requests.get(url).content)
        self.insert1(dict(key, flower_image=local_file))     

In [32]:
StateFlower().populate()

/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/alaska.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/alabama.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/arkansas.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/arizona.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/california.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/colorado.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/connecticut.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/delaware.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/florida.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/georgia.jpg
/home/dimitri/dev/db-programming-with-datajoint/notebooks/stage/states/flowers/ha

In [ ]:
dj.Diagram(schema)